In [8]:
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from IPython import display
import random
import scipy.stats
from sklearn import metrics
from sklearn.metrics import roc_curve
import os
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster## Fidelity Analysis
from sklearn.cluster import KMeans

import timm
import torchvision
from torchvision.transforms.functional import gaussian_blur
from torch.utils.data import Subset
import torch.nn.functional as F
import matplotlib.pyplot as plt

# CIFAR-10

In [148]:
data='CIFAR10'
num_experiments=5
n_rows=5000
sigma_list=[0.1,0.25]
k_list=[50,100, 1000]
m=10
epochs=30
epsilon=0.0
clipping_mode='nonDP'
model='vit_small_patch16_224'
explanation_type = 'sl'

df_gap_nondp = pd.DataFrame()

for sigma in sigma_list:
    for k in k_list:
        gap_dir = f'perturbation_gap_pred_test/{data}_sigma={sigma}_k={k}_m={m}'
        df_path = f'{gap_dir}/mode={clipping_mode}_nrows={n_rows}_type={explanation_type}_epochs={epochs}.csv'
        if not os.path.exists(df_path):
            continue
        df_one_setting = pd.read_csv(df_path)
#             df_one_setting['expl_type'] = explanation_type
        df_gap_nondp = pd.concat([df_gap_nondp, df_one_setting], ignore_index=True)

df_gap_nondp['epsilon'] = np.inf
df_gap_nondp_grouped = df_gap_nondp.groupby(['m', 'k', 'sigma'])
df_gap_nondp_grouped[['gap']].agg(['mean', 'std'])

gap          
                       mean       std
m    k      sigma                    
10.0 50.0   0.10   0.023692  0.004005
            0.25   0.082453  0.016574
     100.0  0.10   0.023449  0.004925
            0.25   0.082711  0.017934
     1000.0 0.10   0.022582  0.004872
            0.25   0.078223  0.015414

In [149]:
clipping_mode='BK-MixOpt'

epsilons = [0.5, 1.0, 2.0, 8.0]

df_gap_dp = pd.DataFrame()
for sigma in sigma_list:
    for k in k_list:
        for eps in epsilons:
            gap_dir = f'perturbation_gap_pred_test/{data}_sigma={sigma}_k={k}_m={m}'
            df_dir = f'{gap_dir}/mode={clipping_mode}_nrows={n_rows}_type={explanation_type}_epsilon={eps}_epochs={epochs}.csv'
            if not os.path.exists(df_dir):
                continue
            df_one_eps = pd.read_csv(df_dir)
#                 df_one_eps['expl_type'] = explanation_type
            df_gap_dp = pd.concat([df_gap_dp, df_one_eps], ignore_index=True)
    
df_gap_dp_grouped = df_gap_dp.groupby(['k', 'sigma', 'epsilon'])
df_gap_dp_grouped[['gap']].agg(['mean', 'std'])

gap          
                          mean       std
k      sigma epsilon                    
50.0   0.10  0.5      0.065437  0.005568
             1.0      0.067329  0.012328
             2.0      0.059255  0.004600
             8.0      0.065145  0.007052
       0.25  0.5      0.245269  0.044353
             1.0      0.250014  0.054672
             2.0      0.232455  0.035113
             8.0      0.227056  0.033146
100.0  0.10  0.5      0.065845  0.006255
             1.0      0.067249  0.012769
             2.0      0.059959  0.004360
             8.0      0.065040  0.007392
       0.25  0.5      0.245458  0.043838
             1.0      0.247787  0.054794
             2.0      0.231291  0.034294
             8.0      0.228091  0.032612
1000.0 0.10  0.5      0.064378  0.006189
             1.0      0.065324  0.011977
             2.0      0.057446  0.003863
             8.0      0.062590  0.006719
       0.25  0.5      0.241932  0.042581
             1.0      0.245755  0.054772
             2.0      0.228080  0.035313
             8.0      0.223925  0.033177

In [150]:
df_gap_combined = pd.concat([df_gap_nondp, df_gap_dp], ignore_index=True)
# df_gap_combined = df_gap_combined.loc
df_gap_combined_grouped = df_gap_combined.groupby(['k', 'sigma', 'epsilon'])
latexify = df_gap_combined_grouped[['gap']].agg(['mean', 'std'])
latexify

gap          
                          mean       std
k      sigma epsilon                    
50.0   0.10  0.5      0.065437  0.005568
             1.0      0.067329  0.012328
             2.0      0.059255  0.004600
             8.0      0.065145  0.007052
             inf      0.023692  0.004005
       0.25  0.5      0.245269  0.044353
             1.0      0.250014  0.054672
             2.0      0.232455  0.035113
             8.0      0.227056  0.033146
             inf      0.082453  0.016574
100.0  0.10  0.5      0.065845  0.006255
             1.0      0.067249  0.012769
             2.0      0.059959  0.004360
             8.0      0.065040  0.007392
             inf      0.023449  0.004925
       0.25  0.5      0.245458  0.043838
             1.0      0.247787  0.054794
             2.0      0.231291  0.034294
             8.0      0.228091  0.032612
             inf      0.082711  0.017934
1000.0 0.10  0.5      0.064378  0.006189
             1.0      0.065324  0.011977
             2.0      0.057446  0.003863
             8.0      0.062590  0.006719
             inf      0.022582  0.004872
       0.25  0.5      0.241932  0.042581
             1.0      0.245755  0.054772
             2.0      0.228080  0.035313
             8.0      0.223925  0.033177
             inf      0.078223  0.015414

In [151]:
print(latexify.to_latex(float_format="{:.3f}".format))

\begin{tabular}{lllrr}
\toprule
 &  &  & \multicolumn{2}{r}{gap} \\
 &  &  & mean & std \\
k & sigma & epsilon &  &  \\
\midrule
\multirow[t]{10}{*}{50.000000} & \multirow[t]{5}{*}{0.100000} & 0.500000 & 0.065 & 0.006 \\
 &  & 1.000000 & 0.067 & 0.012 \\
 &  & 2.000000 & 0.059 & 0.005 \\
 &  & 8.000000 & 0.065 & 0.007 \\
 &  & inf & 0.024 & 0.004 \\
\cline{2-5}
 & \multirow[t]{5}{*}{0.250000} & 0.500000 & 0.245 & 0.044 \\
 &  & 1.000000 & 0.250 & 0.055 \\
 &  & 2.000000 & 0.232 & 0.035 \\
 &  & 8.000000 & 0.227 & 0.033 \\
 &  & inf & 0.082 & 0.017 \\
\cline{1-5} \cline{2-5}
\multirow[t]{10}{*}{100.000000} & \multirow[t]{5}{*}{0.100000} & 0.500000 & 0.066 & 0.006 \\
 &  & 1.000000 & 0.067 & 0.013 \\
 &  & 2.000000 & 0.060 & 0.004 \\
 &  & 8.000000 & 0.065 & 0.007 \\
 &  & inf & 0.023 & 0.005 \\
\cline{2-5}
 & \multirow[t]{5}{*}{0.250000} & 0.500000 & 0.245 & 0.044 \\
 &  & 1.000000 & 0.248 & 0.055 \\
 &  & 2.000000 & 0.231 & 0.034 \\
 &  & 8.000000 & 0.228 & 0.033 \\
 &  & inf & 0.083 &

# SVHN

In [133]:
data='SVHN'
num_experiments=5
n_rows=5000
sigma_list=[0.1,0.25]
k_list=[50,100,1000]
m=10
epochs=50
epsilon=0.0
clipping_mode='nonDP'
model='vit_small_patch16_224'
explanation_type='sl'

df_gap_nondp = pd.DataFrame()

for sigma in sigma_list:
    for k in k_list:
        gap_dir = f'perturbation_gap_test/{data}_sigma={sigma}_k={k}_m={m}'
        df_path = f'{gap_dir}/mode={clipping_mode}_nrows={n_rows}_type={explanation_type}_epochs={epochs}.csv'
        if not os.path.exists(df_path):
            print(df_path)
            continue
        df_one_setting = pd.read_csv(df_path)
        df_gap_nondp = pd.concat([df_gap_nondp, df_one_setting], ignore_index=True)

df_gap_nondp['epsilon'] = np.inf
df_gap_nondp_grouped = df_gap_nondp.groupby(['m', 'k', 'sigma'])
df_gap_nondp_grouped[['gap']].agg(['mean', 'std'])

gap          
                       mean       std
m    k      sigma                    
10.0 50.0   0.10   0.062493  0.011066
            0.25   0.169580  0.029045
     100.0  0.10   0.061709  0.011461
            0.25   0.168471  0.029466
     1000.0 0.10   0.053734  0.010093
            0.25   0.149644  0.024569

In [134]:
clipping_mode='BK-MixOpt'

epsilons = [0.5, 1.0, 2.0, 8.0]

df_gap_dp = pd.DataFrame()
for sigma in sigma_list:
    for k in k_list:
        for eps in epsilons:
            gap_dir = f'perturbation_gap_test/{data}_sigma={sigma}_k={k}_m={m}'
            df_dir = f'{gap_dir}/mode={clipping_mode}_nrows={n_rows}_type={explanation_type}_epsilon={eps}_epochs={epochs}.csv'
            if not os.path.exists(df_dir):
                continue
            df_one_eps = pd.read_csv(df_dir)
            df_gap_dp = pd.concat([df_gap_dp, df_one_eps], ignore_index=True)
    
df_gap_dp_grouped = df_gap_dp.groupby(['k', 'sigma', 'epsilon'])
df_gap_dp_grouped[['gap']].agg(['mean', 'std'])

gap          
                          mean       std
k      sigma epsilon                    
50.0   0.10  0.5      0.770728  0.133268
             1.0      0.733784  0.106286
             2.0      0.756063  0.106511
             8.0      0.749714  0.129884
       0.25  0.5      1.407484  0.096482
             1.0      1.470687  0.128673
             2.0      1.473971  0.150654
             8.0      1.488659  0.140441
100.0  0.10  0.5      0.770840  0.133650
             1.0      0.732568  0.104052
             2.0      0.752734  0.106128
             8.0      0.747161  0.128105
       0.25  0.5      1.406602  0.095693
             1.0      1.469892  0.127981
             2.0      1.472580  0.150050
             8.0      1.487594  0.141127
1000.0 0.10  0.5      0.750518  0.134519
             1.0      0.706339  0.103862
             2.0      0.721509  0.104933
             8.0      0.720538  0.124802
       0.25  0.5      1.394814  0.100875
             1.0      1.458268  0.127691
             2.0      1.459651  0.153925
             8.0      1.472107  0.145638

In [135]:
df_gap_combined = pd.concat([df_gap_nondp, df_gap_dp], ignore_index=True)
# df_gap_combined = df_gap_combined.loc
df_gap_combined_grouped = df_gap_combined.groupby(['k', 'sigma', 'epsilon'])
latexify_svhn = df_gap_combined_grouped[['gap']].agg(['mean', 'std'])
latexify_svhn

gap          
                          mean       std
k      sigma epsilon                    
50.0   0.10  0.5      0.770728  0.133268
             1.0      0.733784  0.106286
             2.0      0.756063  0.106511
             8.0      0.749714  0.129884
             inf      0.062493  0.011066
       0.25  0.5      1.407484  0.096482
             1.0      1.470687  0.128673
             2.0      1.473971  0.150654
             8.0      1.488659  0.140441
             inf      0.169580  0.029045
100.0  0.10  0.5      0.770840  0.133650
             1.0      0.732568  0.104052
             2.0      0.752734  0.106128
             8.0      0.747161  0.128105
             inf      0.061709  0.011461
       0.25  0.5      1.406602  0.095693
             1.0      1.469892  0.127981
             2.0      1.472580  0.150050
             8.0      1.487594  0.141127
             inf      0.168471  0.029466
1000.0 0.10  0.5      0.750518  0.134519
             1.0      0.706339  0.103862
             2.0      0.721509  0.104933
             8.0      0.720538  0.124802
             inf      0.053734  0.010093
       0.25  0.5      1.394814  0.100875
             1.0      1.458268  0.127691
             2.0      1.459651  0.153925
             8.0      1.472107  0.145638
             inf      0.149644  0.024569

In [136]:
print(latexify_svhn.to_latex(float_format="{:.3f}".format))

\begin{tabular}{lllrr}
\toprule
 &  &  & \multicolumn{2}{r}{gap} \\
 &  &  & mean & std \\
k & sigma & epsilon &  &  \\
\midrule
\multirow[t]{10}{*}{50.000000} & \multirow[t]{5}{*}{0.100000} & 0.500000 & 0.771 & 0.133 \\
 &  & 1.000000 & 0.734 & 0.106 \\
 &  & 2.000000 & 0.756 & 0.107 \\
 &  & 8.000000 & 0.750 & 0.130 \\
 &  & inf & 0.062 & 0.011 \\
\cline{2-5}
 & \multirow[t]{5}{*}{0.250000} & 0.500000 & 1.407 & 0.096 \\
 &  & 1.000000 & 1.471 & 0.129 \\
 &  & 2.000000 & 1.474 & 0.151 \\
 &  & 8.000000 & 1.489 & 0.140 \\
 &  & inf & 0.170 & 0.029 \\
\cline{1-5} \cline{2-5}
\multirow[t]{10}{*}{100.000000} & \multirow[t]{5}{*}{0.100000} & 0.500000 & 0.771 & 0.134 \\
 &  & 1.000000 & 0.733 & 0.104 \\
 &  & 2.000000 & 0.753 & 0.106 \\
 &  & 8.000000 & 0.747 & 0.128 \\
 &  & inf & 0.062 & 0.011 \\
\cline{2-5}
 & \multirow[t]{5}{*}{0.250000} & 0.500000 & 1.407 & 0.096 \\
 &  & 1.000000 & 1.470 & 0.128 \\
 &  & 2.000000 & 1.473 & 0.150 \\
 &  & 8.000000 & 1.488 & 0.141 \\
 &  & inf & 0.168 &

# CIFAR100

In [145]:
data='CIFAR100'
num_experiments=5
n_rows=5000
sigma_list=[0.1,0.25]
k_list=[50,100, 1000]
m=10
epochs=9
epsilon=0.0
clipping_mode='nonDP'
model='beit_base_patch16_224.in22k_ft_in22k_in1k'
explanation_type='sl'

df_gap_nondp = pd.DataFrame()

for sigma in sigma_list:
    for k in k_list:
        gap_dir = f'perturbation_gap/{data}_sigma={sigma}_k={k}_m={m}'
        df_path = f'{gap_dir}/mode={clipping_mode}_nrows={n_rows}_type={explanation_type}_epochs={epochs}.csv'
        if not os.path.exists(df_path):
#             print('oops')
            continue
        df_one_setting = pd.read_csv(df_path)
        df_gap_nondp = pd.concat([df_gap_nondp, df_one_setting], ignore_index=True)

df_gap_nondp['epsilon'] = np.inf
df_gap_nondp_grouped = df_gap_nondp.groupby(['m', 'k', 'sigma'])
df_gap_nondp_grouped[['gap']].agg(['mean', 'std'])

gap          
                      mean       std
m    k     sigma                    
10.0 50.0  0.10   0.070382  0.003926
           0.25   0.276722  0.036081
     100.0 0.10   0.069428  0.003769
           0.25   0.275301  0.035744

In [146]:
clipping_mode='BK-MixOpt'

epsilons = [0.5, 1.0, 2.0, 8.0]

df_gap_dp = pd.DataFrame()
for sigma in sigma_list:
    for k in k_list:
        for eps in epsilons:
            gap_dir = f'perturbation_gap/{data}_sigma={sigma}_k={k}_m={m}'
            df_dir = f'{gap_dir}/mode={clipping_mode}_nrows={n_rows}_type={explanation_type}_epsilon={eps}_epochs={epochs}.csv'
            if not os.path.exists(df_dir):
                continue
            df_one_eps = pd.read_csv(df_dir)
            df_gap_dp = pd.concat([df_gap_dp, df_one_eps], ignore_index=True)
    
df_gap_dp_grouped = df_gap_dp.groupby(['k', 'sigma', 'epsilon'])
df_gap_dp_grouped[['gap']].agg(['mean', 'std'])

gap          
                         mean       std
k     sigma epsilon                    
50.0  0.10  0.5      0.131020  0.006569
            1.0      0.208004  0.013746
            2.0      0.319919  0.011432
            8.0      0.404755  0.017115
      0.25  0.5      0.213961  0.007014
            1.0      0.356175  0.014012
            2.0      0.595072  0.017195
            8.0      0.848497  0.036878
100.0 0.10  0.5      0.130827  0.006542
            1.0      0.207983  0.013740
            2.0      0.319602  0.011390
            8.0      0.403514  0.016931
      0.25  0.5      0.213815  0.006954
            1.0      0.356173  0.014270
            2.0      0.594950  0.016962
            8.0      0.847721  0.037792

In [147]:
df_gap_combined = pd.concat([df_gap_nondp, df_gap_dp], ignore_index=True)
# df_gap_combined = df_gap_combined.loc
df_gap_combined_grouped = df_gap_combined.groupby(['k', 'sigma', 'epsilon'])
latexify_cif100 = df_gap_combined_grouped[['gap']].agg(['mean', 'std'])
latexify_cif100

gap          
                         mean       std
k     sigma epsilon                    
50.0  0.10  0.5      0.131020  0.006569
            1.0      0.208004  0.013746
            2.0      0.319919  0.011432
            8.0      0.404755  0.017115
            inf      0.070382  0.003926
      0.25  0.5      0.213961  0.007014
            1.0      0.356175  0.014012
            2.0      0.595072  0.017195
            8.0      0.848497  0.036878
            inf      0.276722  0.036081
100.0 0.10  0.5      0.130827  0.006542
            1.0      0.207983  0.013740
            2.0      0.319602  0.011390
            8.0      0.403514  0.016931
            inf      0.069428  0.003769
      0.25  0.5      0.213815  0.006954
            1.0      0.356173  0.014270
            2.0      0.594950  0.016962
            8.0      0.847721  0.037792
            inf      0.275301  0.035744

In [144]:
print(latexify_cif100.to_latex(float_format="{:.3f}".format))

\begin{tabular}{lllrr}
\toprule
 &  &  & \multicolumn{2}{r}{gap} \\
 &  &  & mean & std \\
k & sigma & epsilon &  &  \\
\midrule
\multirow[t]{10}{*}{50.000000} & \multirow[t]{5}{*}{0.100000} & 0.500000 & 0.131 & 0.007 \\
 &  & 1.000000 & 0.208 & 0.014 \\
 &  & 2.000000 & 0.320 & 0.011 \\
 &  & 8.000000 & 0.404 & 0.017 \\
 &  & inf & 0.071 & 0.004 \\
\cline{2-5}
 & \multirow[t]{5}{*}{0.250000} & 0.500000 & 0.214 & 0.007 \\
 &  & 1.000000 & 0.356 & 0.014 \\
 &  & 2.000000 & 0.595 & 0.017 \\
 &  & 8.000000 & 0.848 & 0.038 \\
 &  & inf & 0.277 & 0.036 \\
\cline{1-5} \cline{2-5}
\multirow[t]{10}{*}{100.000000} & \multirow[t]{5}{*}{0.100000} & 0.500000 & 0.131 & 0.007 \\
 &  & 1.000000 & 0.208 & 0.014 \\
 &  & 2.000000 & 0.319 & 0.012 \\
 &  & 8.000000 & 0.404 & 0.017 \\
 &  & inf & 0.070 & 0.004 \\
\cline{2-5}
 & \multirow[t]{5}{*}{0.250000} & 0.500000 & 0.214 & 0.007 \\
 &  & 1.000000 & 0.356 & 0.014 \\
 &  & 2.000000 & 0.595 & 0.017 \\
 &  & 8.000000 & 0.848 & 0.037 \\
 &  & inf & 0.277 &

## Fidelity Analysis - Hierarchical

In [12]:
data='CIFAR10'
num_experiments=5
n_rows=10000
channel=1
epochs=30
epsilon=0.0
clipping_mode='nonDP'
model='vit_small_patch16_224'
explanation_type='ig'
clustering_method='hierarchical'
fid_method='all_clusters'

df_nondp = pd.read_csv(f'fidelity/fid_{data}_{clustering_method}_method={fid_method}/mode={clipping_mode}_nrows={n_rows}_type={explanation_type}_epochs={epochs}_channel={channel}.csv')
    
df_nondp_grouped = df_nondp.groupby(['max_distance'])
df_nondp_grouped[['consistency', 'num_clusters']].agg(['mean', 'std'])

consistency           num_clusters           
                    mean       std         mean        std
max_distance                                              
0.3             0.188596  0.065455          2.2   0.836660
0.4             0.235840  0.034403         11.0   4.062019
0.5             0.232262  0.035899         30.8  11.562872
0.6             0.215622  0.020133         54.6  10.014989
0.7             0.206074  0.020136         83.4  15.453155

In [13]:
clipping_mode='BK-MixOpt'

epsilons = [0.5, 1.0, 2.0, 8.0]

df_dp = pd.DataFrame()
for eps in epsilons:
    df_one_eps = pd.read_csv(
        f'fidelity/fid_{data}_{clustering_method}_method={fid_method}/mode={clipping_mode}_nrows={n_rows}_type={explanation_type}_epsilon={eps}_epochs={epochs}_channel={channel}.csv')

    df_dp = pd.concat([df_dp, df_one_eps], ignore_index=True)
df_dp_grouped = df_dp.groupby(['max_distance', 'epsilon'])
print('Consistency Data, DP')
df_dp_grouped[['consistency', 'num_clusters']].agg(['mean', 'std'])

Consistency Data, DP


consistency           num_clusters           
                            mean       std         mean        std
max_distance epsilon                                              
1.5          0.5        0.375661  0.156409          2.4   1.140175
             1.0        0.420606  0.193860          2.2   1.643168
             2.0        0.324518  0.200772          1.4   0.547723
             8.0        0.254359  0.113154          1.8   0.836660
2.0          0.5        0.326589  0.093082         12.0   4.847680
             1.0        0.291613  0.068553          9.8   3.271085
             2.0        0.272771  0.033485          8.0   1.870829
             8.0        0.280474  0.051869         11.4   2.880972
2.5          0.5        0.257817  0.036265         33.4   7.300685
             1.0        0.248362  0.037188         30.8   6.496153
             2.0        0.221077  0.025623         24.0   1.870829
             8.0        0.240371  0.030833         28.4   6.503845
3.0          0.5        0.239758  0.026585         68.4  11.104053
             1.0        0.225020  0.008285         63.0   6.964194
             2.0        0.203125  0.020925         53.4  13.939153
             8.0        0.210944  0.011502         52.2   7.155418
3.5          0.5        0.207073  0.018249         94.2   9.391486
             1.0        0.209710  0.004271         97.0   8.396428
             2.0        0.200362  0.015719         84.4  18.460769
             8.0        0.207383  0.023414         78.0   8.000000
4.0          0.5        0.186383  0.016365        124.4  13.277801
             1.0        0.192659  0.013448        120.0   6.324555
             2.0        0.183444  0.011479        108.2  13.645512
             8.0        0.185881  0.008820        109.2  11.903781

## Fidelity Analysis - K-Means

In [16]:
data='CIFAR10'
num_experiments=5
n_rows=10000
channel=1
epochs=30
epsilon=0.0
clipping_mode='nonDP'
model='vit_small_patch16_224'
explanation_type='ig'
clustering_method='kmeans'
fid_method='max'

df_nondp = pd.read_csv(f'fidelity/fid_{data}_{clustering_method}_method={fid_method}/mode={clipping_mode}_nrows={n_rows}_type={explanation_type}_epochs={epochs}_channel={channel}.csv')
    
df_nondp_grouped = df_nondp.groupby(['k'])
df_nondp_grouped[['consistency', 'num_clusters']].agg(['mean', 'std'])

consistency           num_clusters          
             mean       std         mean       std
k                                                 
25.0     0.107753  0.008183          3.2  2.774887
50.0     0.125218  0.007672          3.8  1.788854
100.0    0.122129  0.010654          4.2  1.483240
150.0    0.127879  0.005073          4.6  1.140175
200.0    0.124560  0.008919          5.8  2.774887
250.0    0.124768  0.013068          5.6  2.190890

In [17]:
clipping_mode='BK-MixOpt'

epsilons = [0.5, 1.0, 2.0, 8.0]

df_dp = pd.DataFrame()
for eps in epsilons:
    df_one_eps = pd.read_csv(
        f'fidelity/fid_{data}_{clustering_method}_method={fid_method}/mode={clipping_mode}_nrows={n_rows}_type={explanation_type}_epsilon={eps}_epochs={epochs}_channel={channel}.csv')

    df_dp = pd.concat([df_dp, df_one_eps], ignore_index=True)
df_dp_grouped = df_dp.groupby(['k', 'epsilon'])
print('Consistency Data, DP')
df_dp_grouped[['consistency', 'num_clusters']].agg(['mean', 'std'])

Consistency Data, DP


consistency           num_clusters          
                     mean       std         mean       std
k     epsilon                                             
25.0  0.5        0.134484  0.025108          3.8  1.788854
      1.0        0.122771  0.011132          3.0  1.414214
      2.0        0.114240  0.009952          2.2  0.836660
      8.0        0.113569  0.012186          3.0  1.414214
50.0  0.5        0.139760  0.011722          4.2  1.643168
      1.0        0.140354  0.018084          4.8  0.836660
      2.0        0.114613  0.011699          3.4  1.816590
      8.0        0.122000  0.010789          4.2  1.788854
100.0 0.5        0.142444  0.014243          4.2  1.788854
      1.0        0.131847  0.020391          3.8  1.303840
      2.0        0.128767  0.018730          3.0  1.414214
      8.0        0.126103  0.020807          4.4  2.302173
150.0 0.5        0.133862  0.019616          3.4  1.140175
      1.0        0.135675  0.012573          4.4  1.673320
      2.0        0.119139  0.013119          2.4  0.547723
      8.0        0.130805  0.015131          4.2  2.167948
200.0 0.5        0.149168  0.017666          6.2  2.774887
      1.0        0.144598  0.024483          5.0  1.581139
      2.0        0.130440  0.008670          3.8  1.303840
      8.0        0.119514  0.013853          4.6  1.516575
250.0 0.5        0.145741  0.014150          5.6  1.516575
      1.0        0.132595  0.019682          5.8  2.588436
      2.0        0.133377  0.013403          5.8  2.387467
      8.0        0.132200  0.015720          4.8  1.643168

## Fidelity Generation with k-means

In [8]:
# NEW CODE, TRY IT OUT
num_experiments=5
epochs=[5,10,30]
epochs_nondp = [5,10,30]
model="vit_small_patch16_224"
clipping_mode_dp='BK-MixOpt'
clipping_mode_nondp='nonDP'
num_samples=20000
cifar_data='CIFAR10'
expl_type='sl'
ep=10 # 10 epochs (for now)
# n_rows=100
n_rows=5000
channel=1

df_fid = pd.DataFrame(columns=['experiment_no', 'k', 'consistency', 'num_clusters'])
for experiment_no in tqdm(range(5)):
    exp_no = str(experiment_no)
    df_path = f'lira/attributions_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_type={expl_type}_nsamples=20_epochs={ep}/channel{channel}/{exp_no}.csv'
    # randomly sample only 1000 examples to read in
    skip_rows = np.random.choice(np.arange(0, 20000), size=19000, replace=False)
    print(skip_rows)
    df_channel1 = pd.read_csv(df_path,
#                               skiprows=skip_rows,
#                               header=None
                              nrows=n_rows
                             )
    indices_path = f'lira/indices_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}/{exp_no}.csv'
    preds_path = f'lira/preds_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_type={expl_type}_nsamples=20_epochs={ep}/{exp_no}.csv'
    df_preds = pd.read_csv(preds_path,
#                            skiprows=skip_rows,
#                            header=None
                           nrows=n_rows
                          )
    df_indices = pd.read_csv(indices_path,
#                              skiprows=skip_rows,
#                              header=None
                             nrows=n_rows
                            )
    k_list = [5, 10, 50, 100]
    for k in tqdm(k_list):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(df_channel1)
#         linkage_matrix = linkage(df_channel1, method='complete')
#         clusters = fcluster(linkage_matrix, dist, criterion='distance')
        clusters = kmeans.labels_
        df_preds[str(k)] = clusters
        consistency=0.0
        total_in_clusters=0.0
        large_cluster_count = 0
        for c in np.unique(clusters):
            df_preds_subset = df_preds.loc[df_preds[str(k)] == c]
#             mode_label = df_preds_subset[exp_no].mode().iloc[0]
#             mode_label_count = len(df_preds_subset.loc[df_preds_subset[exp_no] == mode_label])
            total_cluster_size = len(df_preds_subset)
            if total_cluster_size <= 1.0:
                continue
            cluster_counts = df_preds_subset[exp_no].value_counts().tolist()
            for cluster_count in cluster_counts:
                consistency += cluster_count * (cluster_count - 1) / (total_cluster_size - 1)
            total_in_clusters += total_cluster_size
#             if total_cluster_size != 1.0:
#                 print(total_cluster_size, mode_label_count / total_cluster_size)
#                 consistency+=mode_label_count
#                 total_in_clusters+=total_cluster_size
#                 large_cluster_count += 1
        if total_in_clusters > 2.0:
            row = {
                'experiment_no': experiment_no,
                'k': k,
                'consistency': consistency / total_in_clusters,
                'num_clusters': large_cluster_count
            }
            print(f'consistency for exp {experiment_no}, k {k}, num_clusters {large_cluster_count}: {consistency / total_in_clusters}')
            df_fid.loc[len(df_fid)] = list(row.values())
            
# df_fid.to_csv(f'fidelity/fid_kmeans_{clipping_mode_nondp}_{cifar_data}_{n_rows}_type={expl_type}_epochs={ep}_channel={channel}.csv')
df_fid.head(60)

KeyboardInterrupt: 

In [ ]:
num_experiments=5
epochs=[5,10,30]
epochs_nondp = [5,10,30]
model="vit_small_patch16_224"
clipping_mode_dp='BK-MixOpt'
# clipping_mode_nondp='nonDP'
num_samples=20000
cifar_data='CIFAR10'
expl_type='ixg'
ep=10 # 10 epochs (for now)
epsilons=[0.5, 1.0, 2.0, 8.0]
# n_rows=100
n_rows=5000
channel=1

df_fid = pd.DataFrame(columns=['experiment_no', 'k', 'consistency', 'num_clusters', 'epsilon'])

for eps in epsilons:
    for experiment_no in tqdm(range(5)):
        exp_no = str(experiment_no)
        df_path = f'lira/attributions_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={expl_type}_nsamples=20_epochs={ep}/channel{channel}/{exp_no}.csv'
        # randomly sample only 1000 examples to read in
        skip_rows = np.random.choice(np.arange(0, 20000), size=19000, replace=False)
#         print(skip_rows)
        df_channel1 = pd.read_csv(df_path,
    #                               skiprows=skip_rows,
    #                               header=None
                                  nrows=n_rows
                                 )
        indices_path = f'lira/indices_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}/{exp_no}.csv'
        preds_path = f'lira/preds_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={expl_type}_nsamples=20_epochs={ep}/{exp_no}.csv'
        df_preds = pd.read_csv(preds_path,
    #                            skiprows=skip_rows,
    #                            header=None
                               nrows=n_rows
                              )
        df_indices = pd.read_csv(indices_path,
    #                              skiprows=skip_rows,
    #                              header=None
                                 nrows=n_rows
                                )
        k_list = [5, 10, 50, 100]
        for k in tqdm(k_list):
            kmeans = KMeans(n_clusters=k)
            kmeans.fit(df_channel1)
    #         linkage_matrix = linkage(df_channel1, method='complete')
    #         clusters = fcluster(linkage_matrix, dist, criterion='distance')
            clusters = kmeans.labels_
            df_preds[str(k)] = clusters
            consistency=0.0
            total_in_clusters=0.0
            large_cluster_count = 0
            for c in np.unique(clusters):
                df_preds_subset = df_preds.loc[df_preds[str(k)] == c]
                mode_label = df_preds_subset[exp_no].mode().iloc[0]
                mode_label_count = len(df_preds_subset.loc[df_preds_subset[exp_no] == mode_label])
                total_cluster_size = len(df_preds_subset)
                if total_cluster_size != 1.0:
                    print(total_cluster_size, mode_label_count / total_cluster_size)
                    consistency+=mode_label_count
                    total_in_clusters+=total_cluster_size
                    large_cluster_count += 1
            if total_in_clusters > 2.0:
                row = {
                    'experiment_no': experiment_no,
                    'k': k,
                    'consistency': consistency / total_in_clusters,
                    'num_clusters': large_cluster_count,
                    'epsilon': eps,
                }
                print(f'consistency for exp {experiment_no}, k {k}, eps {eps}, num_clusters {large_cluster_count}: {consistency / total_in_clusters}')
                df_fid.loc[len(df_fid)] = list(row.values())

df_fid.to_csv(f'fidelity/fid_kmeans_{clipping_mode_dp}_{cifar_data}_{n_rows}_type={expl_type}_epochs={ep}_channel={channel}.csv')
df_fid.head(60)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

/n/home05/czhuangusa/miniconda3/envs/cnn/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


4996 0.1088871096877502
consistency for exp 0, k 5, eps 0.5, num_clusters 1: 0.1088871096877502


/n/home05/czhuangusa/miniconda3/envs/cnn/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


...

/n/home05/czhuangusa/miniconda3/envs/cnn/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


2 0.5
4990 0.10501002004008016
consistency for exp 2, k 10, eps 2.0, num_clusters 2: 0.10516826923076923


/n/home05/czhuangusa/miniconda3/envs/cnn/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


2 0.5
5 0.4
4946 0.10473109583501819
consistency for exp 2, k 50, eps 2.0, num_clusters 3: 0.10518877448011306


/n/home05/czhuangusa/miniconda3/envs/cnn/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


## Fidelity generation with hierarchical clustering

In [4]:
num_experiments=5
epochs=[5,10,30]
epochs_nondp = [5,10,30]
model="vit_small_patch16_224"
clipping_mode_dp='BK-MixOpt'
clipping_mode_nondp='nonDP'
num_samples=20000
cifar_data='CIFAR10'
expl_type='ixg'
ep=10 # 10 epochs (for now)
# n_rows=100
n_rows=5000
channel=1

df_fid = pd.DataFrame(columns=['experiment_no', 'max_distance', 'consistency', 'num_clusters'])
for experiment_no in tqdm(range(5)):
    exp_no = str(experiment_no)
    df_path = f'lira/attributions_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_type={expl_type}_nsamples=20_epochs={ep}/channel{channel}/{exp_no}.csv'
    # randomly sample only 1000 examples to read in
    skip_rows = np.random.choice(np.arange(0, 20000), size=19000, replace=False)
    print(skip_rows)
    df_channel1 = pd.read_csv(df_path,
#                               skiprows=skip_rows,
#                               header=None
                              nrows=n_rows
                             )
    indices_path = f'lira/indices_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_epochs={ep}/{exp_no}.csv'
    preds_path = f'lira/preds_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_nondp}_type={expl_type}_nsamples=20_epochs={ep}/{exp_no}.csv'
    df_preds = pd.read_csv(preds_path,
#                            skiprows=skip_rows,
#                            header=None
                           nrows=n_rows
                          )
    df_indices = pd.read_csv(indices_path,
#                              skiprows=skip_rows,
#                              header=None
                             nrows=n_rows
                            )
    max_distances = [0.1, 0.2, 0.3, 0.4]
    for dist in tqdm(max_distances):
        linkage_matrix = linkage(df_channel1, method='complete')
        clusters = fcluster(linkage_matrix, dist, criterion='distance')
        df_preds[str(dist)] = clusters
        consistency=0.0
        total_in_clusters=0.0
        large_cluster_count = 0
        for c in np.unique(clusters):
            df_preds_subset = df_preds.loc[df_preds[str(dist)] == c]
            mode_label = df_preds_subset[exp_no].mode().iloc[0]
            mode_label_count = len(df_preds_subset.loc[df_preds_subset[exp_no] == mode_label])
            total_cluster_size = len(df_preds_subset)
            if total_cluster_size != 1.0:
                print(total_cluster_size, mode_label_count / total_cluster_size)
                consistency+=mode_label_count
                total_in_clusters+=total_cluster_size
                large_cluster_count += 1
        if total_in_clusters > 2.0:
            row = {
                'experiment_no': experiment_no,
                'max_distance': dist,
                'consistency': consistency / total_in_clusters,
                'num_clusters': large_cluster_count
            }
            print(f'consistency for exp {experiment_no}, dist {dist}, num_clusters {large_cluster_count}: {consistency / total_in_clusters}')
            df_fid.loc[len(df_fid)] = list(row.values())
            
df_fid.to_csv(f'fidelity/fid_hierarchical_{clipping_mode_nondp}_{cifar_data}_{n_rows}_type={expl_type}_epochs={ep}_channel={channel}.csv')
df_fid.head(60)

  0%|          | 0/5 [00:00<?, ?it/s]

[17069  8229 13765 ...  7719  7226  2651]


  0%|          | 0/4 [00:00<?, ?it/s]

2 0.5
23 0.30434782608695654
2 0.5
2 0.5
3 0.3333333333333333
2 1.0
2 0.5
4 0.5
190 0.3473684210526316
2 1.0
2 0.5
2 0.5
2 0.5
2 0.5
2 0.5
18 0.3888888888888889
2 0.5
consistency for exp 0, dist 0.2, num_clusters 17: 0.3702290076335878
2 0.5
2 1.0
4 0.25
2 0.5
59 0.1694915254237288
2 0.5
2 0.5
2 0.5
2 0.5
3 0.6666666666666666
6 0.5
19 0.2631578947368421
5 0.4
4 0.75
5 0.2
19 0.3157894736842105
45 0.26666666666666666
904 0.23119469026548672
2 1.0
2 0.5
2 1.0
2 0.5
3 0.6666666666666666
3 0.6666666666666666
2 0.5
2 1.0
2 0.5
2 0.5
2 0.5
3 0.3333333333333333
2 0.5
4 0.5
2 0.5
6 0.5
2 0.5
10 0.3
2 0.5
3 0.6666666666666666
4 0.5
2 0.5
3 0.6666666666666666
2 0.5
3 0.3333333333333333
2 1.0
3 0.6666666666666666
consistency for exp 0, dist 0.3, num_clusters 45: 0.2611683848797251
2 0.5
2 1.0
2 1.0
2 0.5
3 0.6666666666666666
2 1.0
11 0.18181818181818182
4 0.5
3 0.3333333333333333
2 0.5
2 0.5
2 0.5
3 0.3333333333333333
2 1.0
2 0.5
9 0.3333333333333333
9 0.3333333333333333
4 0.5
4 0.25
17 0.1764705

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

2 1.0
3 0.3333333333333333
2 0.5
4 0.5
3 0.6666666666666666
2 1.0
2 1.0
3 0.6666666666666666
2 0.5
7 0.42857142857142855
3 0.6666666666666666
2 1.0
2 0.5
2 0.5
243 0.32510288065843623
2 0.5
15 0.26666666666666666
14 0.2857142857142857
34 0.2647058823529412
2 1.0
2 1.0
2 1.0
2 1.0
2 0.5
consistency for exp 3, dist 0.2, num_clusters 24: 0.3641456582633053
2 0.5
2 0.5
6 0.3333333333333333
6 0.16666666666666666
3 0.3333333333333333
397 0.17632241813602015
6 0.5
3 0.6666666666666666
4 0.25
2 0.5
2 0.5
4 0.25
2 0.5
4 0.25
16 0.1875
2 0.5
2 0.5
2 0.5
4 0.5
3 0.6666666666666666
2 0.5
2 0.5
2 1.0
3 0.6666666666666666
2 0.5
2 0.5
10 0.3
2 1.0
2 0.5
2 0.5
3 0.3333333333333333
2 1.0
2 0.5
2 0.5
2 0.5
2 0.5
2 1.0
2 0.5
2 1.0
2 0.5
2 0.5
4 0.5
4 0.5
7 0.5714285714285714
6 0.8333333333333334
2 0.5
2 1.0
4 0.75
2 0.5
5 1.0
2 0.5
3 0.3333333333333333
3 0.6666666666666666
3 0.3333333333333333
2 1.0
3 0.6666666666666666
709 0.2143864598025388
3 0.3333333333333333
4 0.5
12 0.3333333333333333
consistency f

  0%|          | 0/4 [00:00<?, ?it/s]

28 0.25
2 0.5
consistency for exp 4, dist 0.2, num_clusters 2: 0.26666666666666666
2 0.5
2 0.5
2 0.5
2 0.5
2 1.0
3 0.3333333333333333
2 0.5
3 0.3333333333333333
3 0.6666666666666666
3 0.6666666666666666
3 0.3333333333333333
219 0.2237442922374429
2 1.0
2 0.5
4 0.5
3 0.3333333333333333
7 0.2857142857142857
6 0.3333333333333333
6 0.3333333333333333
2 0.5
2 0.5
2 0.5
2 0.5
30 0.2
24 0.3333333333333333
3 0.3333333333333333
3 0.3333333333333333
2 0.5
3 1.0
3 0.6666666666666666
3 0.6666666666666666
2 1.0
2 1.0
2 0.5
2 1.0
2 0.5
2 0.5
consistency for exp 4, dist 0.3, num_clusters 37: 0.30517711171662126
4 0.75
2 1.0
2 0.5
2 0.5
2 0.5
2 0.5
2 1.0
2 0.5
11 0.36363636363636365
7 0.7142857142857143
2 0.5
2 0.5
2 1.0
2 0.5
2 0.5
2 0.5
2 1.0
3 0.3333333333333333
2 0.5
2 0.5
2 0.5
2 0.5
7 0.42857142857142855
3 0.3333333333333333
3 0.3333333333333333
3 0.3333333333333333
2 0.5
2 0.5
640 0.16875
5 0.6
156 0.17307692307692307
3 0.6666666666666666
2 0.5
2 0.5
2 0.5
2 0.5
2 0.5
4 0.5
8 0.5
2 0.5
2 1.0
4 

,experiment_no,max_distance,consistency,num_clusters
0,0.0,0.2,0.370229,17.0
1,0.0,0.3,0.261168,45.0
2,0.0,0.4,0.206064,60.0
3,1.0,0.2,0.318841,18.0
4,1.0,0.3,0.250821,52.0
5,1.0,0.4,0.188364,67.0
6,2.0,0.1,1.000000,1.0
7,2.0,0.2,0.344937,16.0
8,2.0,0.3,0.233807,42.0
9,2.0,0.4,0.183318,47.0


In [ ]:
num_experiments=5
epochs=[5,10,30]
epochs_nondp = [5,10,30]
model="vit_small_patch16_224"
clipping_mode_dp='BK-MixOpt'
# clipping_mode_nondp='nonDP'
num_samples=20000
cifar_data='CIFAR10'
expl_type='ixg'
ep=10 # 10 epochs (for now)
epsilons=[0.5, 1.0, 2.0, 8.0]
n_rows=5000
channel=1

df_fid = pd.DataFrame(columns=['experiment_no', 'max_distance', 'consistency', 'num_clusters', 'epsilon'])

for eps in epsilons:
    for experiment_no in tqdm(range(5)):
        exp_no = str(experiment_no)
        df_path = f'lira/attributions_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={expl_type}_nsamples=20_epochs={ep}/channel{channel}/{exp_no}.csv'
        # randomly sample only 1000 examples to read in
        skip_rows = np.random.choice(np.arange(0, 20000), size=19000, replace=False)
#         print(skip_rows)
        df_channel1 = pd.read_csv(df_path,
    #                               skiprows=skip_rows,
    #                               header=None
                                  nrows=n_rows
                                 )
        indices_path = f'lira/indices_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_epochs={ep}/{exp_no}.csv'
        preds_path = f'lira/preds_{cifar_data}_{num_samples}/model={model}_mode={clipping_mode_dp}_eps={eps}_type={expl_type}_nsamples=20_epochs={ep}/{exp_no}.csv'
        df_preds = pd.read_csv(preds_path,
    #                            skiprows=skip_rows,
    #                            header=None
                               nrows=n_rows
                              )
        df_indices = pd.read_csv(indices_path,
    #                              skiprows=skip_rows,
    #                              header=None
                                 nrows=n_rows
                                )
        max_distances = [1.0, 1.5, 2.0, 2.5]
        for dist in tqdm(max_distances):
            linkage_matrix = linkage(df_channel1, method='complete')
            clusters = fcluster(linkage_matrix, dist, criterion='distance')
            df_preds[str(dist)] = clusters
            consistency=0.0
            total_in_clusters=0.0
            large_cluster_count = 0
            for c in np.unique(clusters):
                df_preds_subset = df_preds.loc[df_preds[str(dist)] == c]
                mode_label = df_preds_subset[exp_no].mode().iloc[0]
                mode_label_count = len(df_preds_subset.loc[df_preds_subset[exp_no] == mode_label])
                total_cluster_size = len(df_preds_subset)
                if total_cluster_size != 1.0:
                    print(total_cluster_size, mode_label_count / total_cluster_size)
                    consistency+=mode_label_count
                    total_in_clusters+=total_cluster_size
                    large_cluster_count += 1
            if total_in_clusters > 2.0:
                row = {
                    'experiment_no': experiment_no,
                    'max_distance': dist,
                    'consistency': consistency / total_in_clusters,
                    'num_clusters': large_cluster_count,
                    'epsilon': eps,
                }
                print(f'consistency for exp {experiment_no}, dist {dist}, eps {eps}, num_clusters {large_cluster_count}: {consistency / total_in_clusters}')
                df_fid.loc[len(df_fid)] = list(row.values())

df_fid.to_csv(f'fidelity/fid_hierarchical_{clipping_mode_dp}_{cifar_data}_{n_rows}_type={expl_type}_epochs={ep}_channel={channel}.csv')
df_fid.head(60)